In [1]:
from sklearn.datasets import load_iris
import numpy as np
import math
from tqdm import tqdm
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [2]:
model = LogisticRegression(C=0.5,solver='newton-cg')

In [3]:
iden = pd.read_csv('train_identity.csv')

In [4]:
trans = pd.read_csv('train_transaction.csv')

In [5]:
joined = trans.merge(iden,how='left')

In [6]:
del iden

In [7]:
del trans

In [8]:
joined = joined[joined['TransactionDT'].notnull()]

In [9]:
joined = joined[joined['isFraud'].notnull()]

In [10]:
limite = np.percentile(joined['TransactionDT'], 80)

In [11]:
limite

12192853.600000001

In [12]:
filtro1 = joined['TransactionDT'] > limite

In [13]:
filtro1.value_counts()

False    472432
True     118108
Name: TransactionDT, dtype: int64

In [14]:
validation = joined[filtro1]

In [15]:
filtro2 = joined['TransactionDT']<= limite

In [16]:
train = joined[filtro2]

In [17]:
label = train[['isFraud']]

In [18]:
del(train['isFraud'])

In [19]:
del(train['TransactionID'])

In [20]:
entrenando = train.copy()

In [21]:
del joined

In [22]:
dict_one_hot = {}

In [23]:
def one_hot_encode(df, columna, dict_one_hot):
    if columna not in dict_one_hot:
        dict_one_hot[columna] = OneHotEncoder(handle_unknown='ignore')
        dict_one_hot[columna].fit_transform(df[[columna]].astype(str)).astype(int)
    col_encoded = (dict_one_hot[columna].transform(df[[columna]].astype(str)).astype(int))
    col_encoded = pd.DataFrame(col_encoded.todense()).add_prefix(columna + '_').fillna(0)
    del(df[columna])
    df_nuevo = pd.concat([df.reset_index(drop=True), col_encoded], axis=1)
    return df_nuevo

In [24]:
train = one_hot_encode(train,'id_34',dict_one_hot)

In [25]:
train = one_hot_encode(train,'id_35',dict_one_hot)

In [26]:
train = one_hot_encode(train,'id_38',dict_one_hot)

In [27]:
train = one_hot_encode(train,'ProductCD',dict_one_hot)

In [28]:
train = one_hot_encode(train,'card4',dict_one_hot)

In [29]:
train = one_hot_encode(train,'P_emaildomain',dict_one_hot)

In [30]:
le = {} #genero diccionario por columna para tener un labelEnc por col
for column in train.columns:
    if train[column].dtypes==object:
        le[column] = OrdinalEncoder(handle_unknown='ignore')
        train[column] = le[column].fit_transform(train[column].astype(str).values.reshape(-1, 1))

In [31]:
train.fillna(-1,inplace=True)

In [34]:
model.fit(train, label)

C:\Users\julip\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(C=0.67, max_iter=1000, random_state=0, solver='saga',
                   tol=0.01)

In [35]:
print(classification_report(label, model.predict(train)))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    455833
           1       0.11      0.00      0.00     16599

    accuracy                           0.96    472432
   macro avg       0.54      0.50      0.49    472432
weighted avg       0.93      0.96      0.95    472432



In [36]:
roc_auc_score(label, model.predict_proba(train)[:,1])

0.500128756490462

### VALIDATION

In [36]:
validation.head(2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,C,9300,103.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN
472433,3459433,0,12192911,52.811,C,8809,179.0,106.0,visa,137.0,...,chrome generic for android,NaN,NaN,NaN,F,F,T,F,mobile,F3311


In [37]:
label2=validation['isFraud']

In [38]:
del(validation['isFraud'])

In [39]:
del(validation['TransactionID'])

In [40]:
validation = one_hot_encode(validation,'id_34',dict_one_hot)

In [41]:
validation = one_hot_encode(validation,'id_35',dict_one_hot)

In [42]:
validation = one_hot_encode(validation,'id_38',dict_one_hot)

In [43]:
validation = one_hot_encode(validation,'ProductCD',dict_one_hot)

In [44]:
validation = one_hot_encode(validation,'card4',dict_one_hot)

In [45]:
validation = one_hot_encode(validation,'P_emaildomain',dict_one_hot)

In [46]:
for column in validation.columns:
    if validation[column].dtype==object:
        validation[column] = le[column].transform(validation[column].astype(str).values.reshape(-1, 1))


In [47]:
validation.fillna(-1,inplace=True)

In [49]:
print(classification_report(label2, model.predict(validation)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    114044
           1       0.00      0.00      0.00      4064

    accuracy                           0.97    118108
   macro avg       0.48      0.50      0.49    118108
weighted avg       0.93      0.97      0.95    118108



C:\Users\julip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\julip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\julip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
roc_auc_score(label2, model.predict_proba(validation)[:,1])

0.5284647807055821

### TEST

In [ ]:
idenTest = pd.read_csv('test_identity.csv')

In [ ]:
transTest = pd.read_csv('test_transaction.csv')

In [ ]:
idenTest.columns

In [ ]:
def renombrar(df):
    for columna in df.columns:
        if columna.startswith('id'):
            ss = columna.split('-')
            df.rename(columns={columna:ss[0]+'_'+ss[1]},inplace=True)
    return (df)
        

In [ ]:
idenTest = renombrar(idenTest)

In [ ]:
idenTest.columns

In [ ]:
test = transTest.merge(idenTest,how='left')

In [ ]:
ids = test[['TransactionID']]

In [ ]:
del(test['TransactionID'])

In [ ]:
test.head(2)

In [ ]:
test = one_hot_encode(test,'id_34',dict_one_hot)

In [ ]:
test = one_hot_encode(test,'id_35',dict_one_hot)

In [ ]:
test = one_hot_encode(test,'id_38',dict_one_hot)

In [ ]:
test = one_hot_encode(test,'ProductCD',dict_one_hot)

In [ ]:
test = one_hot_encode(test,'P_emaildomain',dict_one_hot)

In [ ]:
test = one_hot_encode(test,'card4',dict_one_hot)

In [ ]:
for column in test.columns:
    if test[column].dtype==object:
        test[column] = le[column].transform(test[column].astype(str).values.reshape(-1, 1))

In [ ]:
test.fillna(-1,inplace=True)

In [ ]:
test.head(2)

In [ ]:
sub = pd.read_csv('sample_submission.csv')

In [ ]:
sub.head(2)

In [ ]:
test['predict'] = model.predict_proba(test)[:,1]

In [ ]:
test = test[['predict']]

In [ ]:
join = ids.join(test,how='outer')

In [ ]:
join

PENALTY

TOL

C

SOLVER

SCORING:AUCROC

penalty: , 'l1'

In [21]:
entrenando.ProductCD.value_counts()

W    346001
C     56410
R     32203
H     29691
S      8127
Name: ProductCD, dtype: int64

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
logistic = LogisticRegression(tol=1e-2, max_iter=1000,
                              random_state=0)
distributions = dict(C=[0.1,0.2,0.12,0.5,0.67,0.8,1],
                     penalty=['l2','l1'], solver=['lbfgs', 'saga', 'liblinear', 'sag'])
clf = RandomizedSearchCV(logistic, distributions, scoring='roc_auc',n_jobs=1, verbose=100, cv=3,n_iter=4, random_state=0)
search = clf.fit(train, label)
search.best_params_

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3; 1/4] START C=0.8, penalty=l1, solver=saga..............................


C:\Users\julip\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 1/3; 1/4] END ............C=0.8, penalty=l1, solver=saga; total time=14.1min
[CV 2/3; 1/4] START C=0.8, penalty=l1, solver=saga..............................


C:\Users\julip\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 2/3; 1/4] END ............C=0.8, penalty=l1, solver=saga; total time= 9.3min
[CV 3/3; 1/4] START C=0.8, penalty=l1, solver=saga..............................


...

[CV 2/3; 4/4] END .......C=0.5, penalty=l2, solver=liblinear; total time= 1.4min
[CV 3/3; 4/4] START C=0.5, penalty=l2, solver=liblinear.........................


C:\Users\julip\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 3/3; 4/4] END .......C=0.5, penalty=l2, solver=liblinear; total time= 1.5min


C:\Users\julip\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\julip\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.8}

In [33]:
import pickle
with open('model_backup', 'wb') as model_backup_file:
    pickle.dump(search, model_backup_file)

In [34]:
print(classification_report(label, search.predict(train)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    455833
           1       0.30      0.01      0.03     16599

    accuracy                           0.96    472432
   macro avg       0.63      0.51      0.50    472432
weighted avg       0.94      0.96      0.95    472432



In [35]:
roc_auc_score(label, search.predict_proba(train)[:,1])

0.7344186927679233

In [48]:
print(classification_report(label2, search.predict(validation)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    114044
           1       0.31      0.02      0.03      4064

    accuracy                           0.96    118108
   macro avg       0.64      0.51      0.51    118108
weighted avg       0.94      0.96      0.95    118108



In [49]:
roc_auc_score(label2, search.predict_proba(validation)[:,1])

0.7469679237113068

In [50]:
"""import pickle
with open('model_backup', 'rb') as file:
    search = pickle.load(file)"""

"import pickle\nwith open('model_backup', 'rb') as file:\n    search = pickle.load(file)"

In [33]:
model = search.best_estimator_

In [ ]:
params = search.best_params_